In [2]:
import pandas as pd

Preliminary data changes

In [3]:
df = pd.read_csv("product_sales.csv")
# Drop random columns
df = df.drop(columns=["Unnamed: 107", "94"])
# Melt data and remove strings for week names
df = df.melt(id_vars=["Scode", "Pcode", "Price"], var_name="Week", value_name="Quantity Sold")
df["Week"] = df["Week"].str.extract('(\d+)').astype(int)

<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\d'
/var/folders/hh/rn_fsj015zz6cmlmssx4w0y00000gn/T/ipykernel_35371/2419203732.py:6: SyntaxWarning: invalid escape sequence '\d'
  df["Week"] = df["Week"].str.extract('(\d+)').astype(int)


Interpolate data: for each date present in the dataset, we create a data point for each SKU when it doesn’t exist– it will use the same quantity and unit price as the most recent date where that SKU has a data point. 
Use time series (ARIMA model) to forecast future dates. Use these as validation data for the hyperparameter search. This way we are making optimal policies for the most recent data, and calculating our loss based on forecasted data.


# Use Arima for short term interpolation

In [19]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import ray

In [12]:
df

,Scode,Pcode,Price,Week,Quantity Sold
0,Store1,SKU1,$24.00,0,23
1,Store1,SKU2,$63.00,0,23
2,Store1,SKU3,$93.00,0,7
3,Store1,SKU4,$48.00,0,50
4,Store1,SKU5,$62.00,0,8
...,...,...,...,...,...
623995,Store5,SKU1809,$71.00,103,39
623996,Store5,SKU1563,$64.00,103,24
623997,Store5,SKU897,$77.00,103,28
623998,Store5,SKU1142,$81.00,103,31


In [13]:
# Prepare data for Sarima

# Will contain (store, sku) pair
all_unique_combos = []
for store in df["Scode"].unique():
    holder = df[df["Scode"] == store]
    skus = holder["Pcode"].unique()
    for sku in skus:
        all_unique_combos.append((store, sku))


In [17]:
df[(df["Pcode"] == "SKU1") & (df["Scode"] == "Store1")].reset_index(drop=True)

,Scode,Pcode,Price,Week,Quantity Sold
0,Store1,SKU1,$24.00,0,23
1,Store1,SKU1,$24.00,1,48
2,Store1,SKU1,$24.00,2,47
3,Store1,SKU1,$24.00,3,42
4,Store1,SKU1,$24.00,4,22
...,...,...,...,...,...
99,Store1,SKU1,$24.00,99,133
100,Store1,SKU1,$24.00,100,229
101,Store1,SKU1,$24.00,101,381
102,Store1,SKU1,$24.00,102,21


In [21]:
# Parellelize this forecasting for speed
# @ray.remote
def forecast_dates(df, indentifier_pair):
    # Get the current time series based on identifier pair
    current_ts = df[((df["Scode"] == indentifier_pair[0]) & df["Pcode"] == indentifier_pair[1])].reset_index(drop=True)
    # Convert it into a pandas series with regular index
    data = current_ts["Quantity Sold"]
    model = SARIMAX(data, order=(1, 1, 1), seasonal_order=(1, 1, 1, 52))
    results = model.fit()
    forecast = results.get_forecast(steps=52)
    forecast_summary = forecast.summary_frame(alpha=0.05)
    return forecast, forecast_summary

In [22]:
forecast, forecast_summary = forecast_dates(df, ("Store1", "SKU1"))

/Users/elizaknapp/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/Users/elizaknapp/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/Users/elizaknapp/miniconda3/lib/python3.12/site-packages/statsmodels/base/model.py:534: RuntimeWarning: invalid value encountered in scalar divide
  return -self.loglike(params, *args) / nobs
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=          NaN    |proj g|=          NaN


LinAlgError: Schur decomposition solver error.